In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator
import numpy as np
import os

# File paths - replace with your actual file paths
compound_files = [
    '../data/docking_results/Final_Dataset_DB.csv',
    '../data/docking_results/Final_Dataset_EN.csv',
    '../data/docking_results/Final_Dataset_NP.csv'
]

docking_files = [
    '../data/docking_results/3ebh.csv',
    '../data/docking_results/3i65.csv',
    '../data/docking_results/8em8.csv',
    '../data/docking_results/9nsr.csv'
]

def load_compounds(files):
    dfs = []
    for file in files:
        df = pd.read_csv(file)
        dfs.append(df)
    compounds = pd.concat(dfs, ignore_index=True)
    return compounds[['ID', 'smiles']]

def load_and_merge_docking(files):
    dfs = []
    for file in files:
        protein_code = os.path.splitext(os.path.basename(file))[0]
        df = pd.read_csv(file)
        df['ID'] = df['ID'].str.removeprefix('lig_')
        df['ID'] = df['ID'].str.removesuffix('.pdbqt')
        df = df[['ID', 'Binding_Affinity']]
        df = df.rename(columns={'Binding_Affinity': f'Binding_Affinity_{protein_code}'})
        dfs.append(df)
    # Merge docking results on compound_id
    docking_merged = dfs[0]
    for df in dfs[1:]:
        docking_merged = docking_merged.merge(df, on='ID', how='inner')
    return docking_merged

morgan_gen = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=2048)

def smiles_to_fingerprint(smiles):
    if not isinstance(smiles, str):
        return None
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    fp = morgan_gen.GetFingerprint(mol)
    arr = np.zeros((2048,), dtype=int)
    fp.ToBitString()  # optional, just to check
    # Convert to numpy array
    from rdkit.DataStructs import ConvertToNumpyArray
    ConvertToNumpyArray(fp, arr)
    return arr

# Load compounds
compounds = load_compounds(compound_files)

# Load and merge docking scores
docking_scores = load_and_merge_docking(docking_files)

# Merge compounds and docking on compound_id
df = compounds.merge(docking_scores, on='ID', how='inner')

# Convert SMILES to fingerprints
fingerprints = []
for smi in df['smiles']:
    fp = smiles_to_fingerprint(smi)
    if fp is None:
        # Handle invalid SMILES by skipping or imputing zero vector
        fp = np.zeros(2048, dtype=int)
    fingerprints.append(fp)
fingerprints = np.array(fingerprints)

# Prepare features (X) and targets (y)
X = fingerprints
protein_codes = ['3ebh', '3i65', '8em8', '9nsr']  
y = df[[f'Binding_Affinity_{code}' for code in protein_codes]].values.astype(float)


print("Feature matrix shape:", X.shape)
print("Target matrix shape:", y.shape)

# Save or return processed arrays as needed for model training
# For example, save as numpy files:
np.save('../data/docking_results/X_features.npy', X)
np.save('../data/docking_results/y_docking_scores.npy', y)


Feature matrix shape: (318967, 2048)
Target matrix shape: (318967, 4)


In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

# Assume X, y are your features and multi-target docking scores numpy arrays

# Split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define model
input_layer = Input(shape=(X.shape[1],))
x = Dense(128, activation='relu')(input_layer)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
output_layer = Dense(y.shape[1], activation='linear')(x)

model = Model(inputs=input_layer, outputs=output_layer)

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mse'])

# Train model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=32,
    verbose=1
)

# Save model
model.save("../models/docking_score_prediction_model.h5")


Epoch 1/100
7975/7975 ━━━━━━━━━━━━━━━━━━━━ 283s 30ms/step - loss: 0.8812 - mse: 0.8812 - val_loss: 0.5889 - val_mse: 0.5889
Epoch 2/100
7975/7975 ━━━━━━━━━━━━━━━━━━━━ 222s 27ms/step - loss: 0.5016 - mse: 0.5016 - val_loss: 0.4987 - val_mse: 0.4987
Epoch 3/100
7975/7975 ━━━━━━━━━━━━━━━━━━━━ 147s 18ms/step - loss: 0.4905 - mse: 0.4905 - val_loss: 0.4972 - val_mse: 0.4972
Epoch 4/100
7975/7975 ━━━━━━━━━━━━━━━━━━━━ 124s 15ms/step - loss: 0.4833 - mse: 0.4833 - val_loss: 0.5037 - val_mse: 0.5037
Epoch 5/100
7975/7975 ━━━━━━━━━━━━━━━━━━━━ 87s 11ms/step - loss: 0.4775 - mse: 0.4775 - val_loss: 0.5361 - val_mse: 0.5361
Epoch 6/100
7975/7975 ━━━━━━━━━━━━━━━━━━━━ 150s 19ms/step - loss: 0.4713 - mse: 0.4713 - val_loss: 0.4947 - val_mse: 0.4947
Epoch 7/100
7975/7975 ━━━━━━━━━━━━━━━━━━━━ 142s 18ms/step - loss: 0.4663 - mse: 0.4663 - val_loss: 0.5028 - val_mse: 0.5028
Epoch 8/100
7975/7975 ━━━━━━━━━━━━━━━━━━━━ 141s 18ms/step - loss: 0.4619 - mse: 0.4619 - val_loss: 0.4997 - val_mse: 0.4997
Epoch 9/1

In [1]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt

def evaluate_model_performance(y_true, y_pred, protein_columns):
    for i, col_name in enumerate(protein_columns):
        mse = mean_squared_error(y_true[:, i], y_pred[:, i])
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_true[:, i], y_pred[:, i])
        r2 = r2_score(y_true[:, i], y_pred[:, i])
        print(f"Metrics for {col_name}:")
        print(f"  RMSE: {rmse:.4f}")
        print(f"  MAE : {mae:.4f}")
        print(f"  R²  : {r2:.4f}")
        print("-" * 40)
        # Plot true vs predicted
        plt.figure(figsize=(6,6))
        plt.scatter(y_true[:, i], y_pred[:, i], alpha=0.5)
        lims = [min(y_true[:, i].min(), y_pred[:, i].min()),
                max(y_true[:, i].max(), y_pred[:, i].max())]
        plt.plot(lims, lims, 'r--')
        plt.xlabel('True Docking Score')
        plt.ylabel('Predicted Docking Score')
        plt.title(f'{col_name} - Predicted vs True')
        plt.grid(True)
        plt.show()

# Example usage:
# y_val - your true docking score matrix (num_samples x num_proteins)
# model - your trained Keras/TensorFlow model
# X_val - your validation features matrix

# Your exact docking score column names
protein_columns = [
    'Binding_affinity_3ebh',
    'Binding_affinity_3i65',
    'Binding_affinity_8em8',
    'Binding_affinity_9nsr'
]

# Predict on validation data
y_pred = model.predict(X_val)

# Evaluate
evaluate_model_performance(y_val, y_pred, protein_columns)


NameError: name 'model' is not defined

In [ ]:
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor

# docking score columns
protein_columns = [
    'Binding_affinity_3ebh',
    'Binding_affinity_3i65',
    'Binding_affinity_8em8',
    'Binding_affinity_9nsr'
]

# Split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 1. Define base regressor and wrap in MultiOutputRegressor
base_rf = RandomForestRegressor(n_estimators=100, random_state=42)
multi_rf = MultiOutputRegressor(base_rf)

# 2. Train model
multi_rf.fit(X_train, y_train)

# 3. Save each underlying model separately
for i, model in enumerate(multi_rf.estimators_):
    filename = '../models/f"rf_{protein_columns[i]}.joblib"'
    joblib.dump(model, filename)
    print(f"Saved model for {protein_columns[i]} at {filename}")

# 4. Evaluate on validation set
y_pred = multi_rf.predict(X_val)

def evaluate_model_performance(y_true, y_pred, protein_columns):
    for i, col_name in enumerate(protein_columns):
        mse = mean_squared_error(y_true[:, i], y_pred[:, i])
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_true[:, i], y_pred[:, i])
        r2 = r2_score(y_true[:, i], y_pred[:, i])
        print(f"Metrics for {col_name}:")
        print(f"  RMSE: {rmse:.4f}")
        print(f"  MAE : {mae:.4f}")
        print(f"  R²  : {r2:.4f}")
        print("-" * 40)
        # Scatter plot true vs predicted
        plt.figure(figsize=(6,6))
        plt.scatter(y_true[:, i], y_pred[:, i], alpha=0.6)
        lims = [min(y_true[:, i].min(), y_pred[:, i].min()),
                max(y_true[:, i].max(), y_pred[:, i].max())]
        plt.plot(lims, lims, 'r--')
        plt.xlabel('True Docking Score')
        plt.ylabel('Predicted Docking Score')
        plt.title(f'{col_name} - Predicted vs True')
        plt.grid(True)
        plt.show()

evaluate_model_performance(y_val, y_pred, protein_columns)


: 